C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_30412\2987830005.py:7: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='ISO-8859-1')


'tofdrug_cohort_sample100.csv'